In [ ]:
import os
import glob
import json
import re

import matplotlib.pyplot as plt
import math
import pandas as pd
import numpy as np

import nibabel
import nilearn

from nilearn import image as nimg
from nilearn import plotting 
from nilearn.plotting import plot_glass_brain
from nilearn.glm.second_level import SecondLevelModel
from nilearn.masking import intersect_masks
from nilearn.image import math_img
from nilearn.glm import threshold_stats_img


In [ ]:
#get 3 group means

In [ ]:
def get_group_average_contrast(contrast,ses,group,task,covariates,subgroup,rm_CUD,desired_outliers):
    print('Now processing: '+ ' '.join([contrast,ses,group,task]))
    space='MNI152NLin6Asym' 
    
    #check if mriqc summary exists and find runs that need to be excluded based on desired outlier cutoffs
    if desired_outliers:
        if len(glob.glob(f'../../../derivatives/mriqc_summaries/group-{group}_ses-{ses}_task-{task}_rec-unco.tsv'))==0:
            print('No mriqc summary found. Please create this before requesting to remove outliers.')
            return (None, None, None, None)

    excluded_subs_runs_dict = get_excluded_runs(desired_outliers,contrast,ses,group,task)

    effect_size_maps, all_subs = get_effect_size_maps(contrast,ses,group,task,excluded_subs_runs_dict)
    
    
    #remove participants with CUD at baseline (only applies to MM)
    if rm_CUD:
        effect_size_maps, all_subs = rm_CUD_baseline(effect_size_maps, all_subs)

        
    #ensure we have data
    if not effect_size_maps:
        print('No effect size maps, so cannot generate group level output for: '+' '.join([contrast,ses,group,task]))
        return (None, None, None, None)
    
    #note that subject formatting is changed slightly to match that of the non_imaging_data csv file
    #need these to make design matrix
    subs_csv = ['_'.join([s for s in re.split(r'(MM|HC)', sub) if s]) for sub in all_subs]
    
    
    if subgroup:
        effect_size_maps, subs_csv, all_subs = select_subgroup(subgroup, effect_size_maps, subs_csv, all_subs)

    
    #needed for printing total later
    part_count = len(subs_csv)
    
    #create design matrix including all covariates and a way to encode group average (either as col of 1s or with Male/Female)
    group_design_matrix = create_group_design_matrix(subs_csv,group,ses,covariates)
    display(group_design_matrix)
    
    #grab masks needed
    masks = []
    for sub in all_subs:
        masks+=glob.glob(f'../../../derivatives/ses-{ses}/sub-{sub}*/ses-{ses}/func/sub-{sub}*task-{task}*rec-unco*run-*_*{space}*brain_mask.nii.gz')

    #make intersection of the masks
    #threshold=1 corresponds to keeping the intersection of all masks, whereas threshold=0 is the union of all masks
    mask = intersect_masks(masks, threshold=1, connected=True)


    #define model specs
    second_level_model = SecondLevelModel(mask_img=mask, target_affine=None, target_shape=None, 
                                          smoothing_fwhm=None, memory_level=1, 
                                          verbose=0, n_jobs=-2, minimize_memory=False)

    #fit model
    second_level_model = second_level_model.fit(effect_size_maps, design_matrix=group_design_matrix,)

    
    #define contrast matrix 
    contrast_matrix = np.eye(group_design_matrix.shape[1])
    grp_contrasts = dict([(column, contrast_matrix[i])
                      for i, column in enumerate(group_design_matrix.columns)])

    print(grp_contrasts)

    
    contrast_outputs = {}
    
    for key in grp_contrasts.keys():
        #compute the contrasts
        #pick stat_type none because it automatically selects t vs. F test 
        contrast_output = second_level_model.compute_contrast(second_level_contrast=grp_contrasts[key], 
                                                              second_level_stat_type=None, output_type='all')
        contrast_outputs[key] = contrast_output

        
    #for printing the covariates
    if covariates == '':
        covariates = 'nothing'
        
    print('Processing done for: '+' '.join([contrast,ses,group,task]))
    print(f'Controlled for {covariates}')
    
    return (contrast_outputs, mask, part_count, subs_csv)



In [ ]:
def get_excluded_runs(desired_outliers,contrast,ses,group,task):
    
    #empty list and dict to save runs to be excluded
    excluded_runs=[]
    excluded_runs_dict={}
    
    if len(desired_outliers.keys())==0:
        return excluded_runs_dict
    
    #read in cutoffs based on Q1-1.5*IQR and Q3+1.5*IQR based on all tasks and groups (not just current) for this session
    iqr_cutoffs_df = pd.read_csv(f'../../../derivatives/mriqc_summaries/iqr_cutoffs_ses-{ses}_rec-unco.tsv', low_memory=False, sep='\t')
    
    #read in tsv file with mriqc summary for current task, ses, group combi
    mriqc_summary = pd.read_csv(f'../../../derivatives/mriqc_summaries/group-{group}_ses-{ses}_task-{task}_rec-unco.tsv', low_memory=False, sep='\t')
    
    #possible outlier types that use IQR-based cutoff: tsnr, snr, gsr_x, gsr_y
    #exclude runs based on these if in desired_outliers list
    for outlier_type in ['tsnr', 'snr', 'gsr_x', 'gsr_y']:
        if outlier_type in desired_outliers.keys():
            #get upper and lower cutoffs from iqr summaries df
            lower_bound = iqr_cutoffs_df[outlier_type].iloc[0]
            upper_bound = iqr_cutoffs_df[outlier_type].iloc[1]
            
            #find subs_runs combinations outside of cutoffs 
            above_subs_runs = mriqc_summary['subs_runs'][mriqc_summary[outlier_type]>=upper_bound].tolist()
            below_subs_runs = mriqc_summary['subs_runs'][mriqc_summary[outlier_type]<=lower_bound].tolist()
            subs_runs = above_subs_runs+below_subs_runs
            excluded_runs+=subs_runs
            
    if 'fd_mean' in desired_outliers.keys():
        above_fd_mean_runs = mriqc_summary['subs_runs'][mriqc_summary['fd_mean']>desired_outliers['fd_mean']].tolist()
        if 'fd_perc' in desired_outliers:
            below_fd_perc_runs = set(mriqc_summary['subs_runs'][mriqc_summary['fd_perc']<desired_outliers['fd_perc']].tolist())
            above_fd_mean_runs = set(above_fd_mean_runs)
            above_fd_mean_runs = list(above_fd_mean_runs.difference(below_fd_perc_runs))
        excluded_runs+=above_fd_mean_runs
    
    if 'motion_outlier_cutoff' in desired_outliers.keys():
        perc_cutoff =  desired_outliers['motion_outlier_cutoff']
        
        if task == 'nback':
            n_scans = 278
        elif task == 'mid':
            n_scans = 215
        elif task == 'sst':
            n_scans = 257
        
        motion_outlier_df = pd.read_csv(f'../../../derivatives/motion_outlier_counts/{task}_motion_outlier_count.csv', low_memory=False)
        motion_outlier_above_cutoff_runs = motion_outlier_df['subs_runs'][motion_outlier_df['motion_outlier_count']>(n_scans*perc_cutoff)].tolist()
    
        excluded_runs+=motion_outlier_above_cutoff_runs
    
    
    #remove duplicates
    excluded_runs=list(set(excluded_runs))
    
    #turn into dict
    for sub_run in excluded_runs:
        sub = sub_run.split('_')[0]
        run = sub_run.split('_')[1]
        if sub in excluded_runs_dict.keys():
            excluded_runs_dict[sub].append(run)
        else:
            excluded_runs_dict[sub]=[run]
    
    return excluded_runs_dict
    

In [ ]:
def rm_CUD_baseline(effect_size_maps, all_subs):
    
    #subs to be excluded (only MM) because they had cannabis use disorder at baseline (exclusion criterium)
    excluded_subs = ['MM014','MM188','MM197','MM217','MM228','MM239','MM241']
    
    #get only effect size maps that aren't those of any of the excluded subjects
    final_effect_size_maps = [path for path in effect_size_maps if path.split('/sub-')[1].split('/')[0] not in excluded_subs]

    #get only subjects that aren't those of any of the excluded subjects
    final_all_subs = [sub for sub in all_subs if sub not in excluded_subs]

    return final_effect_size_maps, final_all_subs
    

In [ ]:
def select_subgroup(subgroup, effect_size_maps, subs_csv, subs):
    non_img_data = pd.read_csv(f"../../../sourcedata/non_imaging_data/MMJ-Processed_data-2022_05_27-13_58-6858bbe.csv",low_memory=False)
    final_subs = []
    final_subs_csv = []
    final_effect_size_maps = []
    if subgroup == 'dipstick_THC':
        dipstick_THC_results_dict = non_img_data[non_img_data['SSS.CHR.Time_point'] == 'One year'].groupby('IDS.CHR.Subject')['URN.LGC.THC_present'].agg("first").to_dict()
        for i in range(len(effect_size_maps)):
            if subs_csv[i] in dipstick_THC_results_dict.keys():
                if dipstick_THC_results_dict[subs_csv[i]]:
                    final_subs.append(subs[i])
                    final_subs_csv.append(subs_csv[i])
                    final_effect_size_maps.append(effect_size_maps[i])
                
    return (final_effect_size_maps, final_subs_csv, final_subs)

In [ ]:
def get_effect_size_maps(contrast,ses,group,task,excluded_subs_runs_dict):
        
    #get individual effect size maps
    #get all second-level contrasts that exist
    second_level_effect_size_maps = glob.glob(f'../../../derivatives/task_analysis_volume/second_level/sub-{group}*/ses-{ses}/task-{task}/sub-{group}*_ses-{ses}_task-{task}_rec-unco_contrast-{contrast}_effect_size_fx.nii.gz')
    second_level_subs = [path.split('/sub-')[1].split('/')[0] for path in second_level_effect_size_maps if path]
    second_level_dict = dict(zip(second_level_subs, second_level_effect_size_maps))

    #remove any second level contrasts who have at least one run that needs to be excluded 
    for sub in excluded_subs_runs_dict.keys():
        if sub in second_level_dict.keys():
            second_level_dict.pop(sub)
    
    second_level_subs = list(second_level_dict.keys())
    second_level_effect_size_maps = list(second_level_dict.values())
    
    print(f'second level data subs count: {len(second_level_subs)}')
    
    #get first-level contrasts for the remaining subjects
    all_first_level_effect_size_maps = glob.glob(f'../../../derivatives/task_analysis_volume/first_level/sub-{group}*/ses-{ses}/task-{task}/sub-{group}*_ses-{ses}_task-{task}_rec-unco_run-*_contrast-{contrast}_effect_size.nii.gz')
    first_level_effect_size_maps = [path for path in all_first_level_effect_size_maps if path.split('/sub-')[1].split('/')[0] not in second_level_subs]
    first_level_subs = [path.split('/sub-')[1].split('/')[0] for path in first_level_effect_size_maps if path]
    first_level_runs = [path.split('run-')[1].split('_')[0] for path in first_level_effect_size_maps if path]

    #need to work with sub_run keys since otherwise we'd have duplicate keys
    first_level_subs_runs = ['_'.join(x) for x in zip(first_level_subs,first_level_runs)]
    first_level_dict = dict(zip(first_level_subs_runs, first_level_effect_size_maps))

    for sub,run_list in excluded_subs_runs_dict.items():
        for run in run_list:
            sub_run = f'{sub}_{run}'
            if sub_run in first_level_dict.keys():
                first_level_dict.pop(sub_run)
    
    first_level_subs = [sub_run.split('_')[0] for sub_run in first_level_dict.keys()]
    first_level_effect_size_maps = list(first_level_dict.values())
    
    print(f'first level data subs count: {len(first_level_subs)}')
    
    #combine the effect_size_maps into one list and the subjects into one list
    effect_size_maps = second_level_effect_size_maps + first_level_effect_size_maps
    
    all_subs = second_level_subs + first_level_subs
    
    print(f'all subs count: {len(all_subs)}')
    
    return (effect_size_maps, all_subs)


In [ ]:
def create_group_design_matrix(subs,group,ses,covariates):
    
    df_subs = pd.DataFrame(subs,columns=['subs'])
    non_img_data = pd.read_csv(f"../../../sourcedata/non_imaging_data/MMJ-Processed_data-2022_05_27-13_58-6858bbe.csv",low_memory=False)
    
    #create design matrix with 1 column of 1s and as many rows as there are subjects
    group_design_matrix = pd.DataFrame([1] * len(subs), columns=['group_average'],)
    
    if 'sex' in covariates:
        #add columns for male and female, that will then be combined to create the group average 
        grouped_sex = non_img_data.groupby("IDS.CHR.Subject")["SBJ.CHR.Sex"].agg("first")
        dict_sex = grouped_sex.to_dict()
        df_subs['sex'] = df_subs['subs'].map(dict_sex)
        dummy_df = pd.get_dummies(df_subs['sex'])
        group_design_matrix['sex'] = dummy_df['Male']
    
    #numericals can be added directly
    #mean center numerical values like age and CUDIT!

    if 'age' in covariates:
        #numerical can be added directly
        #mean center numerical value
        grouped_age = non_img_data.groupby("IDS.CHR.Subject")["SBJ.INT.Age"].agg("first")
        dict_age = grouped_age.to_dict()
        group_design_matrix['age'] = df_subs['subs'].map(dict_age)
        group_design_matrix['age'] = group_design_matrix['age'] - group_design_matrix['age'].mean()
    
    if 'total_cudit' in covariates:
        if group == 'HC': 
            grouped_HC_baseline_cudit = non_img_data[non_img_data['SSS.CHR.Time_point'] == 'Screening'].groupby('IDS.CHR.Subject')['INV.INT.CUDIT.Summed_score'].agg("first")
            dict_HC_baseline_cudit = grouped_HC_baseline_cudit.to_dict()
            group_design_matrix['total_cudit'] = df_subs['subs'].map(dict_HC_baseline_cudit)
            group_design_matrix['total_cudit'] = group_design_matrix['total_cudit'] - group_design_matrix['total_cudit'].mean()

        else:
            if ses == 'baseline':
                dict_MM_baseline_cudit = non_img_data[non_img_data['SSS.CHR.Time_point'] == 'Baseline'].groupby('IDS.CHR.Subject')['INV.INT.CUDIT.Summed_score'].agg("first").to_dict()
                group_design_matrix['total_cudit'] = df_subs['subs'].map(dict_MM_baseline_cudit)
                group_design_matrix['total_cudit'] = group_design_matrix['total_cudit'] - group_design_matrix['total_cudit'].mean()
                #needed because MM_141 is nan, so replacing with 0, which is the mean
                group_design_matrix['total_cudit'].fillna(0, inplace=True)
            else:
                dict_MM_1year_cudit = non_img_data[non_img_data['SSS.CHR.Time_point'] == 'One year'].groupby('IDS.CHR.Subject')['INV.INT.CUDIT.Summed_score'].agg("first").to_dict()
                group_design_matrix['total_cudit'] = df_subs['subs'].map(dict_MM_1year_cudit)
                group_design_matrix['total_cudit'] = group_design_matrix['total_cudit'] - group_design_matrix['total_cudit'].mean()
    
    #encode the correspondence 
    freq_dict = {'Once or more per day':7,
        '5-6 days a week':6,
        '3-4 days a week':5,
        '1-2 days a week':4,
        'Less than once a week':3,
        'Less than once every two weeks':2,
        'Less than once a month':1,
        None:0}
    
    if 'THC_freq_month' in covariates:
        if group == 'HC':
            #results from screening visit (using this for consistency since CUDIT-R was also collected at screening visit)
            dict_HC_screening_THC = non_img_data[non_img_data['SSS.CHR.Time_point'] == 'Screening'].groupby('IDS.CHR.Subject')['TLF.CHR.THC.Frequency_in_month'].agg("last").to_dict()
            dict_HC_screening_THC_num = {k:freq_dict[v] for k,v in dict_HC_screening_THC.items()}
            group_design_matrix['THC_freq_month'] = df_subs['subs'].map(dict_HC_screening_THC_num)
            group_design_matrix['THC_freq_month'] = group_design_matrix['THC_freq_month'] - group_design_matrix['THC_freq_month'].mean()
        else:
            if ses == 'baseline':     
                #results from MRI visit (using this for consistency since CUDIT-R was also collected at MRI visit)
                dict_MM_MRIvisit_THC = non_img_data[non_img_data['SSS.CHR.Time_point'] == 'Baseline'].groupby('IDS.CHR.Subject')['TLF.CHR.THC.Frequency_in_month'].agg("first").to_dict()
                dict_MM_MRIvisit_THC_num = {k:freq_dict[v] for k,v in dict_MM_MRIvisit_THC.items()}
                group_design_matrix['THC_freq_month'] = df_subs['subs'].map(dict_MM_MRIvisit_THC_num)
                group_design_matrix['THC_freq_month'] = group_design_matrix['THC_freq_month'] - group_design_matrix['THC_freq_month'].mean()
            else:
                dict_MM_MRIvisit_THC = non_img_data[non_img_data['SSS.CHR.Time_point'] == 'One year'].groupby('IDS.CHR.Subject')['TLF.CHR.THC.Frequency_in_month'].agg("first").to_dict()
                dict_MM_MRIvisit_THC_num = {k:freq_dict[v] for k,v in dict_MM_MRIvisit_THC.items()}
                group_design_matrix['THC_freq_month'] = df_subs['subs'].map(dict_MM_MRIvisit_THC_num)
                group_design_matrix['THC_freq_month'] = group_design_matrix['THC_freq_month'] - group_design_matrix['THC_freq_month'].mean()

                
    return group_design_matrix


In [ ]:
def fdr_threshold_map(z_map,mask):
    threshold_map, threshold = threshold_stats_img(z_map, mask_img=mask, alpha=.05, height_control='fdr', cluster_threshold=0, two_sided=True)
    return (threshold_map, threshold)


In [ ]:
def plot_axial_slices(threshold_map, threshold, MNI_template_used, contrast, group, ses, part_count):

    #display = plotting.plot_stat_map(threshold_map, bg_img = MNI_template_used, display_mode='z', cut_coords=[-28,0,28,40,60],threshold=threshold,vmax=7)
 
    if threshold:
        display = plotting.plot_stat_map(threshold_map, bg_img = MNI_template_used, display_mode='z', cut_coords=[0],vmax=7,threshold=threshold, colorbar=True)
        rounded_thresh = round(threshold,2)
        display.title(text=f'Thresholded z map; "{contrast}" contrast of {group} at {ses}, exp. fdr = .05, z_thresh = {rounded_thresh}, n = {part_count}', size=14, y=1.2)
    else:
        display = plotting.plot_stat_map(threshold_map, bg_img = MNI_template_used, display_mode='z', cut_coords=[0],threshold=None, colorbar=True)
        display.title(text=f'Effect size map; "{contrast}" contrast of {group} at {ses}, n = {part_count}', size=14, y=1.2)
    
    return display


In [ ]:
def plot_glass_brains(threshold_map, threshold, contrast, group, ses, part_count):
    display = plot_glass_brain(
    stat_map_img = threshold_map,
    threshold=threshold,
    colorbar=True,
    display_mode='ortho',
    plot_abs=False)
    rounded_thresh = round(threshold,2)
    display.title(text=f'Z scores on glass brain; "{contrast}" contrast of {group} at {ses}, exp. fdr = .05, z_thresh = {rounded_thresh}, n = {part_count}',
                  size=14, y=1.2)
    return display

In [ ]:
def save_group_level_outputs(standardized_effect_map, stat_map, threshold, group, contrast, task, ses, participants, covariates):
    print(f'Now saving output for group {group} for task {task} and contrast {contrast} in session {ses}')

    
    #check that output does not exist
    output = glob.glob(f'../../../derivatives/task_analysis_volume/group_level/group-{group}/ses-{ses}/task-{task}/group-{group}_ses-{ses}_task-{task}_rec-unco_contrast-{contrast}_std_effect_size.nii.gz')
    if output: #checks if list is not empty
        print(f'At least partial group level output exists for group {group}, session {ses} and task {task}. This must be deleted for before generating new output.')
        return 
    
    
    #create paths to output dir if not exist
    derivatives_path = '../../../derivatives'
    nilearn_output_path = os.path.join(derivatives_path, 'task_analysis_volume','group_level',f'group-{group}',f'ses-{ses}',f'task-{task}')
    if not os.path.isdir(nilearn_output_path):
        os.makedirs (nilearn_output_path)

        
    #save contrast maps to files
    standardized_effect_map.to_filename(f'../../../derivatives/task_analysis_volume/group_level/group-{group}/ses-{ses}/task-{task}/group-{group}_ses-{ses}_task-{task}_rec-unco_contrast-{contrast}_std_effect_size.nii.gz')
    stat_map.to_filename(f'../../../derivatives/task_analysis_volume/group_level/group-{group}/ses-{ses}/task-{task}/group-{group}_ses-{ses}_task-{task}_rec-unco_contrast-{contrast}_z_score.nii.gz')

    with open(f'../../../derivatives/task_analysis_volume/group_level/group-{group}/ses-{ses}/task-{task}/group-{group}_ses-{ses}_task-{task}_rec-unco_contrast-{contrast}_FDR_threshold.txt', 'x') as f:
        f.writelines(str(threshold) + '\n')
    
    with open(f'../../../derivatives/task_analysis_volume/group_level/group-{group}/ses-{ses}/task-{task}/group-{group}_ses-{ses}_task-{task}_rec-unco_contrast-{contrast}_covariates.txt', 'x') as f:
        f.writelines('\n'.join(covariates) + '\n')
        
    with open(f'../../../derivatives/task_analysis_volume/group_level/group-{group}/ses-{ses}/task-{task}/group-{group}_ses-{ses}_task-{task}_rec-unco_contrast-{contrast}_participants.txt', 'x') as f:
        f.writelines('\n'.join(participants) + '\n')
    
    print(f'{group} {task} {ses} {contrast}: processing done')
        

In [ ]:
def get_standardized_effect_map(effect_map,effect_variance_map,N):
    
    #load effect map and variance as arrays
    effect_map_nimg = nimg.load_img(effect_map)
    effect_map_signal = effect_map_nimg.get_fdata(dtype='f4')
    
    effect_variance_map_nimg = nimg.load_img(effect_variance_map)
    effect_variance_map_signal = effect_variance_map_nimg.get_fdata(dtype='f4')
    effect_std_dev_map_signal = np.sqrt(effect_variance_map_signal)
    
    #calculate h to avoid biasing std. effect due to small sample
    h = math.gamma((N-1)/2)/(np.sqrt((N-1)/2)*math.gamma((N-2)/2))
    
    #calculate std effect map
    #avoid division by 0 due to mask
    standardized_effect_map = (np.divide(effect_map_signal, effect_std_dev_map_signal, out=np.zeros_like(effect_map_signal), where=(effect_std_dev_map_signal != 0)))*h
    
    #get the affine matrix
    affine_matrix = effect_map_nimg.affine
    
    #save as nifti again to be able to display below
    standardized_effect_map_nifti = nibabel.Nifti1Image(standardized_effect_map, affine=affine_matrix)

    return standardized_effect_map_nifti


In [ ]:
task='mid'
sessions = ['baseline','1year']
groups = ['HC','MM']
MNI_template_path = '../templates/tpl-MNI152NLin6Asym_res-01_desc-brain_T1w.nii.gz'
MNI_template_used = nibabel.load(MNI_template_path)


if task == 'mid':
        contrasts = [ 'HiRewCue-NeuCue', #high reward anticipation -- paper A2, ABCD
                      'LoRewCue-NeuCue', #low reward anticipation -- ABCD
                      'RewCue-NeuCue', #combined reward anticipation -- paper A1
                      'HiRewCue-LoRewCue', #high vs. low reward anticipation -- paper A3, ABCD
                      'HiRewCue-Baseline', #high reward anticipation vs. baseline -- paper A4
                      'HiLossCue-NeuCue', #high loss anticipation -- paper A5, ABCD
                      'LoLossCue-NeuCue', #low loss anticipation -- ABCD
                      'HiLossCue-LoLossCue', #high vs. low loss anticipation -- ABCD
                      'HiWin-NeuHit', #high reward outcome cp. to neutral hit -- paper O6
                      'Win-NoWin', #combined reward outcome cp. to combined reward miss -- ABCD
                      'HiLoss-NeuHit', #high loss cp. to neutral hit -- paper O7
                      'Loss-AvoidLoss', #combined loss cp. to combined avoid loss -- ABCD
                     ] 
      
elif task == 'sst':
    contrasts=['SuccStop-Go','UnsuccStop-Go','UnsuccStop-SuccStop']
    
elif task == 'nback':
    contrasts=['twoback-zeroback']

#options are age, sex, total_cudit, THC_freq_month at the moment (HC is 0 for total_cudit and THC_freq)
#covariates = ['sex','age','total_cudit', 'THC_freq_month']
covariates = []

#subgroup = 'dipstick_THC'
subgroup = ''

#set to true if MM participants with CUD at baseline should be excluded from the analysis
rm_CUD = True

#set this to the ones we want to include with values for fd_mean and fd_perc as keys
desired_outliers = {'tsnr':[],'snr':[],'gsr_x':[],'gsr_y':[],'fd_mean':0.2,'fd_perc':0.3,'motion_outlier_cutoff':0.3}
#desired_outliers = {}


#loop through all sessions, groups, contrasts
for ses in sessions:
    for group in groups:        
        for contrast in contrasts:
            #get group average group level contrast as the output
            contrast_outputs, mask, part_count, participants = get_group_average_contrast(contrast,ses,group,task,covariates,subgroup,rm_CUD,desired_outliers)
            
            #account for HC not having 1year scans
            if contrast_outputs:
                
                print(f'For task contrast {contrast}, we will print the covariate-based contrasts in the following order: {contrast_outputs.keys()}:')
                
                for slope in contrast_outputs.keys():
                    
                    contrast_output = contrast_outputs[slope]
                    
                    #compute FDR-corrected z-score thresholdmap
                    #note that this does not have a cluster threshold
                    #this image is masked by the mask intersection across all subjects
                    stat_map = contrast_output['z_score']
                    effect_map = contrast_output['effect_size']
                    effect_variance_map = contrast_output['effect_variance']
                    
                    standardized_effect_map = get_standardized_effect_map(effect_map,effect_variance_map,part_count)
                    
                    threshold_map, threshold = fdr_threshold_map(stat_map, mask)
                    
                    #save group average output for fsleyes plotting
                    if slope == 'group_average' and subgroup == '':
                        save_group_level_outputs(standardized_effect_map, stat_map, threshold, group, contrast, task, ses, participants, covariates)
           
                        #make desired plots
                        #mask_axial_slices = plot_axial_slices(mask, None, MNI_template_used, contrast, group, ses, part_count)

                        effect_axial_slices = plot_axial_slices(standardized_effect_map, None, MNI_template_used, contrast, group, ses, part_count)

                        zmap_axial_slices = plot_axial_slices(threshold_map, threshold, MNI_template_used, contrast, group, ses, part_count)

                        #zmap_glass_brain = plot_glass_brains(threshold_map, threshold, contrast, group, ses, part_count)

                
                

In [ ]:
#group difference model for control vs. MM at baseline 

In [ ]:
def get_baseline_group_difference_contrast(contrast,ses,groups,task,covariates,subgroup, rm_CUD,desired_outliers):
    
    #get individual effect size maps and a design matrix per group
    effect_size_maps=[] #list of both groups effect size maps
    group_sizes=[] #list of the two group sizes
    group_design_matrices=[] #get separate design matrices per group that will be combined
    for group in groups:
        excluded_subs_by_group_runs_dict = get_excluded_runs(desired_outliers,contrast,ses,group,task)
        effect_size_maps_by_group, all_subs = get_effect_size_maps(contrast,ses,group,task,excluded_subs_by_group_runs_dict)
        
        #remove participants with CUD at baseline (only applies to MM)
        if rm_CUD:
            effect_size_maps_by_group, all_subs = rm_CUD_baseline(effect_size_maps_by_group, all_subs)
        
        #note that subject formatting is changed slightly to match that of the non_imaging_data csv file
        #need these to make design matrix
        subs_csv = ['_'.join([s for s in re.split(r'(MM|HC)', sub) if s]) for sub in all_subs]
       
        if subgroup:
            effect_size_maps_by_group, subs_csv, all_subs = select_subgroup(subgroup, effect_size_maps_by_group, subs_csv, all_subs)

        effect_size_maps += effect_size_maps_by_group
        group_sizes.append(len(effect_size_maps_by_group))
        
        #create design matrix per group
        #note that the columns that need to stay separate when stacking the design matrices later get renamed
        indv_design_matrix = create_group_design_matrix(subs_csv,group,ses,covariates)
        indv_design_matrix.rename({'group_average': f'group_average_{group}'}, axis='columns',inplace=True)
        
        group_design_matrices.append(indv_design_matrix)
    
    
    #stack design matrices of the two groups to make one large design matrix
    #note again that group_average or Male&Female columns, which were renamed, will not get stacked
    #thus, these will partially contain NaNs, which will be replaced by 0s
    group_design_matrix = pd.concat(group_design_matrices).replace(np. nan,0) 

    print(group_design_matrix)
    
    #grab masks needed (both groups at baseline)
    masks = glob.glob(f'../../../derivatives/ses-{ses}/sub-*/ses-{ses}/func/sub-*task-{task}*rec-unco*run-*_*{space}*brain_mask.nii.gz')

    #make intersection of the masks
    #threshold=1 corresponds to keeping the intersection of all masks, whereas threshold=0 is the union of all masks
    mask = intersect_masks(masks, threshold=1, connected=True)
    
    
    #define group level model
    second_level_model = SecondLevelModel(mask_img=mask, target_affine=None, target_shape=None, 
                                      smoothing_fwhm=None, memory_level=1, 
                                      verbose=0, n_jobs=-2, minimize_memory=False)
    #fit model
    second_level_model = second_level_model.fit(
        effect_size_maps,
        design_matrix=group_design_matrix,
    )

    #make list of possible contrasts: HC only, MM only, MM-HC
    contrast_matrix = np.eye(group_design_matrix.shape[1])
    grp_contrasts = dict([(column, contrast_matrix[i])
                  for i, column in enumerate(group_design_matrix.columns)])
     
    grp_contrasts['group_average'] = grp_contrasts['group_average_MM']-grp_contrasts['group_average_HC']
        
    print(grp_contrasts)
    
    
    contrast_outputs = {}
    
    for key in grp_contrasts.keys():
        if 'group_average_' not in key:
            contrast_output = second_level_model.compute_contrast(second_level_contrast=grp_contrasts[key], 
                                                              second_level_stat_type=None, output_type='all')
            contrast_outputs[key] = contrast_output
    
    #for printing the covariates
    if covariates == '':
        covariates = 'nothing'
        
    print('Processing done of the baseline difference of MM and HC for: '+' '.join([contrast,task]))
    print(f'Controlled for {covariates}')

    
    return (contrast_outputs, mask,(group_sizes[0],group_sizes[1]))



In [ ]:
groups=['MM','HC']
ses='baseline'
task='mid'
space='MNI152NLin6Asym' #change if desired
covariates_list = [[],['sex'],['age'],['THC_freq_month'],['total_cudit'],['age','sex'],['THC_freq_month','age','sex'],['total_cudit','age','sex'] ]
#covariates_list = [[]]
MNI_template_used = nibabel.load('../templates/tpl-MNI152NLin6Asym_res-02_desc-brain_T1w.nii.gz')


if task == 'mid':
        contrasts = [ 'HiRewCue-NeuCue', #high reward anticipation -- paper A2, ABCD
                      'LoRewCue-NeuCue', #low reward anticipation -- ABCD
                      'RewCue-NeuCue', #combined reward anticipation -- paper A1
                      'HiRewCue-LoRewCue', #high vs. low reward anticipation -- paper A3, ABCD
                      'HiRewCue-Baseline', #high reward anticipation vs. baseline -- paper A4
                      'HiLossCue-NeuCue', #high loss anticipation -- paper A5, ABCD
                      'LoLossCue-NeuCue', #low loss anticipation -- ABCD
                      'HiLossCue-LoLossCue', #high vs. low loss anticipation -- ABCD
                      'HiWin-NeuHit', #high reward outcome cp. to neutral hit -- paper O6
                      'Win-NoWin', #combined reward outcome cp. to combined reward miss -- ABCD
                      'HiLoss-NeuHit', #high loss cp. to neutral hit -- paper O7
                      'Loss-AvoidLoss', #combined loss cp. to combined avoid loss -- ABCD
                     ] 
    
elif task == 'sst':
    contrasts=['SuccStop-Go','UnsuccStop-Go','UnsuccStop-SuccStop']

elif task == 'nback':
    contrasts=['twoback-zeroback']

#since we're comparing to HC, setting to only MCC subgroup doesn't make sense   
subgroup = ''

#set to true if MM participants with CUD at baseline should be excluded from the analysis
rm_CUD = True

#set this to the ones we want to include with values for fd_mean and fd_perc as keys
desired_outliers = {'tsnr':[],'snr':[],'gsr_x':[],'gsr_y':[],'fd_mean':0.2,'fd_perc':0.3,'motion_outlier_cutoff':0.3}
#desired_outliers = {}


#TO: loop through all contrasts
for covariates in covariates_list:
    for contrast in contrasts:
        #compute group difference contrast
        contrast_outputs, mask, part_count = get_baseline_group_difference_contrast(contrast,ses,groups,task,covariates,subgroup,rm_CUD,desired_outliers)
        print(covariates)
        
        print(f'We will print the covariate-based contrasts in the following order: {contrast_outputs.keys()}:')

        for key in contrast_outputs.keys():
            
            contrast_output = contrast_outputs[key]
            
            #calculate fdr-thresholded map
            threshold_map, threshold = fdr_threshold_map(contrast_output['z_score'], mask)
            
            if not np.isinf(threshold):
                
                #make visualizations
                fig, ax = plt.subplots()
                fig.text(0.5, 0.5, f"{contrast}, controlled for: {covariates}, showing: {key}", ha='center', va='center', fontsize=16)
                ax.axis('off')
                
                #make visualizations
                axial_slices = plot_axial_slices(threshold_map, threshold, MNI_template_used, contrast, 'MM minus HC', ses, part_count)
                glass_brain = plot_glass_brains(threshold_map, threshold, contrast, 'MM minus HC', ses, part_count)

                #effect_axial_slices = plot_axial_slices(contrast_output['effect_size'], None, MNI_template_used, contrast, group, ses, part_count)



In [ ]:
#PAIRED group difference model for MM at 1year vs. baseline

In [ ]:
def get_MM_paired_group_difference_contrast(contrast,sessions,group,task,covariates,subgroup,rm_CUD,desired_outliers):
        
    #get individual effect size maps by subjects
    effect_size_maps_by_ses = [] #list of two dictionaries, one per ses, both with subjects as keys and their effect size paths as values
    
    for ses in sessions:
        excluded_subs_runs_dict = get_excluded_runs(desired_outliers,contrast,ses,group,task)
        
        effect_size_maps, all_subs = get_effect_size_maps(contrast,ses,group,task,excluded_subs_runs_dict)
        #remove participants with CUD at baseline (only applies to MM)
        if rm_CUD:
            effect_size_maps, all_subs = rm_CUD_baseline(effect_size_maps, all_subs)
        effect_size_by_subj_dict = dict(zip(all_subs, effect_size_maps))
        effect_size_maps_by_ses.append(effect_size_by_subj_dict)

    #find subjects with both baseline and 1year scans
    subjs_with_both_ses = list(set(effect_size_maps_by_ses[0].keys()).intersection(set(effect_size_maps_by_ses[1].keys())))
    
    #subtract baseline from 1year scans
    #save as list of nii effect size maps
    effect_size_maps=[]
    for sub in subjs_with_both_ses:
        effect_size_map_0 = nibabel.load(effect_size_maps_by_ses[0][sub])
        effect_size_map_1 = nibabel.load(effect_size_maps_by_ses[1][sub])
        diff_img = math_img("img0 - img1", img0=effect_size_map_0, img1=effect_size_map_1)
        effect_size_maps.append(diff_img)

    #rewrite subject names so they match the csv file
    subs_csv = ['_'.join([s for s in re.split(r'(MM|HC)', sub) if s]) for sub in subjs_with_both_ses]
    
    if subgroup:
        effect_size_maps, subs_csv, subjs_with_both_ses = select_subgroup(subgroup, effect_size_maps, subs_csv, subjs_with_both_ses)
    
    part_count = len(subjs_with_both_ses)
    
    #get design matrix for the group
    #if sex present, will include Male/Female; otherwise will include a column of all 1s called group average
    group_design_matrix = create_group_design_matrix(subs_csv,group,ses,covariates)

    #grab masks needed (all MM, both 1year and baseline) for the included subjects
    masks = []
    for sub in subjs_with_both_ses:
        masks+=glob.glob(f'../../../derivatives/ses-*/sub-{sub}*/ses-*/func/sub-{group}*task-{task}*rec-unco*run-1_*{space}*brain_mask.nii.gz')

    #make intersection of the masks
    #threshold=1 corresponds to keeping the intersection of all masks, whereas threshold=0 is the union of all masks
    mask = intersect_masks(masks, threshold=1, connected=True)
    
    #define group level model
    #can also take group smoothing out if preferred (helps remove noise for noisier tasks)
    second_level_model = SecondLevelModel(mask_img=mask, target_affine=None, target_shape=None, 
                                      smoothing_fwhm=4.0, memory_level=1, 
                                      verbose=0, n_jobs=-2, minimize_memory=False)
    
    #fit model
    second_level_model = second_level_model.fit(effect_size_maps,design_matrix=group_design_matrix,)
        

    #define contrast matrix to include single column of 1s 
    contrast_matrix = np.eye(group_design_matrix.shape[1])
    grp_contrasts = dict([(column, contrast_matrix[i])
                  for i, column in enumerate(group_design_matrix.columns)])
           
    print(grp_contrasts)
    
    
    contrast_outputs = {}
        
    for key in grp_contrasts.keys():
        #compute all the group contrasts
        contrast_output = second_level_model.compute_contrast(second_level_contrast=grp_contrasts[key], 
                                                              second_level_stat_type=None, output_type='all')
        contrast_outputs[key] = contrast_output
        

    #for printing the covariates
    if covariates == '':
        covariates = 'nothing'
        
    print('Processing done of MM 1year vs. baseline for: '+' '.join([contrast,task]))
    print(f'Controlled for {covariates}')

    return (contrast_outputs, mask, part_count)

    

In [ ]:
sessions=['1year','baseline']
task='sst'
group='MM'
covariates_list = [[],['sex'],['age'],['THC_freq_month'],['total_cudit'],['age','sex'],['THC_freq_month','age','sex'],['total_cudit','age','sex'] ]
#covariates_list = [[]]
space='MNI152NLin6Asym' #change if desired
MNI_template_used = nibabel.load('../templates/tpl-MNI152NLin6Asym_res-02_desc-brain_T1w.nii.gz')


if task == 'mid':
        contrasts = [ 'HiRewCue-NeuCue', #high reward anticipation -- paper A2, ABCD
                      'LoRewCue-NeuCue', #low reward anticipation -- ABCD
                      'RewCue-NeuCue', #combined reward anticipation -- paper A1
                      'HiRewCue-LoRewCue', #high vs. low reward anticipation -- paper A3, ABCD
                      'HiRewCue-Baseline', #high reward anticipation vs. baseline -- paper A4
                      'HiLossCue-NeuCue', #high loss anticipation -- paper A5, ABCD
                      'LoLossCue-NeuCue', #low loss anticipation -- ABCD
                      'HiLossCue-LoLossCue', #high vs. low loss anticipation -- ABCD
                      'HiWin-NeuHit', #high reward outcome cp. to neutral hit -- paper O6
                      'Win-NoWin', #combined reward outcome cp. to combined reward miss -- ABCD
                      'HiLoss-NeuHit', #high loss cp. to neutral hit -- paper O7
                      'Loss-AvoidLoss', #combined loss cp. to combined avoid loss -- ABCD
                     ] 
    
elif task == 'sst':
    contrasts=['SuccStop-Go','UnsuccStop-Go','UnsuccStop-SuccStop']

elif task == 'nback':
    contrasts=['twoback-zeroback']
    
    
subgroup = ''
#subgroup = 'dipstick_THC'

#set to true if MM participants with CUD at baseline should be excluded from the analysis
rm_CUD = True

#set this to the ones we want to include with values for fd_mean and fd_perc as keys
desired_outliers = {'tsnr':[],'snr':[],'gsr_x':[],'gsr_y':[],'fd_mean':0.2,'fd_perc':0.3,'motion_outlier_cutoff':0.3}
#desired_outliers = {}


for covariates in covariates_list:
    for contrast in contrasts:
        contrast_outputs, mask, part_count = get_MM_paired_group_difference_contrast(contrast,sessions,group,task,covariates,subgroup,rm_CUD,desired_outliers)

        print(f'We will print the covariate-based contrasts in the following order: {contrast_outputs.keys()}:')

        for key in contrast_outputs.keys():
            
            contrast_output = contrast_outputs[key]
  
            #calculate fdr-thresholded map
            threshold_map, threshold = fdr_threshold_map(contrast_output['z_score'], mask)
            
            if not np.isinf(threshold):
                
                #make visualizations
                fig, ax = plt.subplots()
                fig.text(0.5, 0.5, f"{contrast}, controlled for: {covariates}, showing: {key}", ha='center', va='center', fontsize=16)
                ax.axis('off')

                axial_slices = plot_axial_slices(threshold_map, threshold, MNI_template_used, contrast, group, '1year minus baseline', part_count)
                glass_brain = plot_glass_brains(threshold_map, threshold, contrast, group, '1year minus baseline', part_count)
                #effect_axial_slices = plot_axial_slices(contrast_output['effect_size'], None, MNI_template_used, contrast, group, '1year minus baseline', part_count)

